In [ ]:
#Load Expression Data

import pandas as pd

# Load expression matrix (tab-separated, quotes present)
expr_data = pd.read_csv("GSE5281_series_matrix.txt", sep="\t", quotechar='"')

# Set the probe ID as index
expr_data.set_index("ID_REF", inplace=True)

# Preview data
expr_data.iloc[:5, :5]


,GSM119615,GSM119616,GSM119617,GSM119618,GSM119619
ID_REF,,,,,
1007_s_at,693.973300,807.145140,1722.625000,1668.057500,2231.29690
1053_at,9.755318,19.052270,13.060389,14.652245,138.67683
117_at,13.346846,18.217276,97.425470,98.197945,106.46844
121_at,173.092450,289.315400,537.182800,496.346560,477.86517
1255_g_at,52.616950,110.375984,99.021736,17.306772,628.81616


In [ ]:
# Load GPL570 annotation and map probe IDs to gene symbols

# Load GPL570 annotation file (skip commented lines)
gpl = pd.read_csv("GPL570-55999.txt", sep="\t", comment="#", low_memory=False)

# Keep only relevant columns
gpl_subset = gpl[["ID", "Gene Symbol"]].dropna()

# Handle cases where multiple gene symbols are listed
gpl_subset["Gene Symbol"] = gpl_subset["Gene Symbol"].apply(lambda x: x.split(" /// ")[0])

# Create a mapping dictionary: Probe ID → Gene Symbol
probe_to_gene = dict(zip(gpl_subset["ID"], gpl_subset["Gene Symbol"]))


In [ ]:
# Map probe IDs to gene symbols and group duplicates

# Copy expression data to avoid modifying original
gene_expr = expr_data.copy()

# Reset index to move 'ID_REF' (probe ID) into a column
gene_expr.reset_index(inplace=True)

# Map probe IDs to gene symbols
gene_expr["Gene Symbol"] = gene_expr["ID_REF"].map(probe_to_gene)

# Drop rows where no gene symbol was found
gene_expr.dropna(subset=["Gene Symbol"], inplace=True)

# Drop the old 'ID_REF' column (probe ID)
gene_expr.drop(columns=["ID_REF"], inplace=True)

# Group by gene symbol and average duplicate probes
gene_expr = gene_expr.groupby("Gene Symbol").mean()

# Show final shape and preview
print("Shape after mapping to gene symbols:", gene_expr.shape)
gene_expr.head()


Shape after mapping to gene symbols: (22880, 161)


,GSM119615,GSM119616,GSM119617,GSM119618,GSM119619,GSM119620,GSM119621,GSM119622,GSM119623,GSM119624,...,GSM238945,GSM238946,GSM238947,GSM238948,GSM238949,GSM238951,GSM238952,GSM238953,GSM238955,GSM238963
Gene Symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,16.092667,34.991314,10.746553,72.359210,109.717970,34.677658,75.050835,40.150936,47.761227,76.548485,...,109.42900,53.012700,45.03970,10.70330,79.8856,18.01250,8.59821,33.20660,112.204000,11.654200
A1BG-AS1,5.116955,67.800510,21.632008,175.544170,79.791710,171.995100,93.985900,228.986600,87.379280,193.311860,...,5.92621,12.746400,10.84530,37.01540,63.8156,43.65630,8.34086,4.38778,11.299100,201.288000
A1CF,13.562965,40.780445,131.987224,55.218665,57.709375,69.129072,112.323790,47.167341,32.731592,43.284466,...,41.40465,2.886165,18.16635,34.83335,76.5694,23.83176,34.69810,8.58483,19.648335,49.599955
A2M,315.357861,133.123514,289.263175,243.749550,940.709050,211.150755,133.338405,161.728978,56.483171,65.616185,...,394.36040,135.051500,550.88100,299.45030,424.3655,129.95650,205.44980,242.84525,761.781700,1418.275000
A2M-AS1,111.023705,80.445760,107.210520,107.729126,178.198700,101.639020,112.686270,76.161950,97.213810,56.894566,...,93.30060,151.017000,178.29400,226.07900,95.1620,90.60830,185.98400,63.23400,224.083000,98.780500


In [14]:
# Noramlize the gene expression data

from sklearn.preprocessing import StandardScaler

# Z-score normalize each gene across samples (rows)
scaler = StandardScaler()
normalized_data = scaler.fit_transform(gene_expr)

# Convert back to DataFrame with same index and columns
normalized_df = pd.DataFrame(normalized_data, index=gene_expr.index, columns=gene_expr.columns)

# Preview
normalized_df.iloc[:5, :5]


,GSM119615,GSM119616,GSM119617,GSM119618,GSM119619
Gene Symbol,,,,,
A1BG,-0.349162,-0.283660,-0.174479,-0.150433,-0.308049
A1BG-AS1,-0.367884,-0.236762,-0.165546,-0.050606,-0.321817
A1CF,-0.353477,-0.275385,-0.074989,-0.167016,-0.331976
A2M,0.161333,-0.143388,0.054071,0.015380,0.074249
A2M-AS1,-0.187226,-0.218687,-0.095321,-0.116214,-0.276545


In [15]:
# Save to CSV
normalized_df.to_csv("GSE5281_normalized_gene_expression.csv")
